In [ ]:
pip install pandas tqdm pillow faiss-cpu

In [ ]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-d0klqtv2
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-d0klqtv2
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [19]:
import os
import json
import urllib.request
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm
from PIL import Image

# Torch & CLIP
import torch
import clip

# Faiss
import faiss

# Metadata Embedding
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [20]:
pd.set_option('display.float_format', '{:.0f}'.format)

In [21]:
import sqlite3
import pandas as pd
import json

conn = sqlite3.connect("/content/drive/MyDrive/real_estate_with_Sgrade.db")
df = pd.read_sql("SELECT * FROM house", conn)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52254 entries, 0 to 52253
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            52254 non-null  int64  
 1   title               52254 non-null  object 
 2   price               52254 non-null  object 
 3   address             52254 non-null  object 
 4   floor               36047 non-null  float64
 5   deposit_type        52254 non-null  object 
 6   management_fee      42877 non-null  float64
 7   available_from      52254 non-null  object 
 8   house_num           52254 non-null  int64  
 9   agent_comm          52078 non-null  float64
 10  agent_info          52254 non-null  object 
 11  rooms_count         52146 non-null  float64
 12  options             52254 non-null  object 
 13  posted_at           51882 non-null  object 
 14  gu                  52254 non-null  object 
 15  dong                52254 non-null  object 
 16  img_

###매물 데이터 불러오기

In [23]:
# img_url 파싱 (JSON 문자열 → 리스트 → 대표(첫 번째) 이미지 추출)
def extract_first_img(json_str):
    try:
        urls = json.loads(json_str)
        return urls[0] if isinstance(urls, list) and urls else None
    except Exception:
        return None

df['img_path'] = df['img_url'].apply(extract_first_img)
df['has_image'] = df['img_path'].notnull()

###이미지 다운로드

In [31]:
os.makedirs("images", exist_ok=True)

In [32]:
from concurrent.futures import ThreadPoolExecutor

def download_and_save(row):
    house_num = int(row['house_num'])
    img_url = row['img_path']
    save_path = f"images/{house_num}.jpg"
    try:
        urllib.request.urlretrieve(img_url, save_path)
        return house_num, save_path
    except:
        return house_num, None

# 멀티스레딩으로 병렬 다운로드
rows = df[df['has_image']].to_dict('records')

with ThreadPoolExecutor(max_workers=16) as executor:
    results = list(tqdm(executor.map(download_and_save, rows), total=len(rows)))

# 결과 반영
for house_num, path in results:
    idx = df[df['house_num'] == house_num].index[0]
    if path:
        df.at[idx, 'local_path'] = path
    else:
        df.at[idx, 'has_image'] = False

100%|██████████| 40080/40080 [02:59<00:00, 223.08it/s]


In [33]:
image_files = os.listdir("images")
print(f"images 폴더에 있는 파일 개수: {len(image_files)}")
print("예시 파일:", image_files[:5])

images 폴더에 있는 파일 개수: 37557
예시 파일: ['2525655220.jpg', '2523157019.jpg', '2526694752.jpg', '2527115391.jpg', '2528235414.jpg']


In [ ]:
# images 폴더에 있는 파일 개수: 37557 -> 중복되는 house_num의 이미지들은 누락됨

In [34]:
failures = [r for r in results if r[1] is None]
print(f"다운로드 실패한 파일 수: {len(failures)}")

다운로드 실패한 파일 수: 0


In [ ]:
# os.makedirs("images", exist_ok=True)

# def download_image(url, path):
#     try:
#         urllib.request.urlretrieve(url, path)
#         return True
#     except:
#         return False

# # 이미지 다운로드
# df['local_path'] = None
# for idx, row in tqdm(df[df['has_image']].iterrows(), total=df['has_image'].sum()):
#     save_path = f"images/{int(row['house_num'])}.jpg"
#     if download_image(row['img_path'], save_path):
#         df.at[idx, 'local_path'] = save_path
#     else:
#         df.at[idx, 'has_image'] = False

  0%|          | 128/40080 [02:04<26:37:54,  2.40s/it]

In [30]:
# # 🚨🚨 이미지 다운 다시 해야될 때 폴더 통째로 날리는 코드
# import shutil
# shutil.rmtree('/content/images')

clip의 역할 : 이미지 -> 임베딩(이미지의 특징을 고차원 숫자 벡터로 변환한 것)

이 임베딩을 벡터 db에 저장해서 faiss를 통해 유사한 값을 가진 이미지를 빠르게 검색함.

###이미지 임베딩 생성 (CLIP)

In [37]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def get_embedding(image_path):
    try:
        image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
        with torch.no_grad():
            vec = model.encode_image(image)
            vec /= vec.norm(dim=-1, keepdim=True)
        return vec[0].cpu().numpy()
    except:
        return None

df['embedding'] = None
for idx, row in tqdm(df[df['has_image']].iterrows(), total=df['has_image'].sum()):
    vec = get_embedding(row['local_path'])
    if vec is not None:
        df.at[idx, 'embedding'] = vec
    else:
        df.at[idx, 'has_image'] = False

100%|██████████| 40080/40080 [09:00<00:00, 74.09it/s]


###메타데이터 벡터 생성

In [38]:
numeric_features = ['deposit', 'monthly_rent', 'space']
categorical_features = ['gu', 'dong']

df[numeric_features] = df[numeric_features].fillna(0)
df[categorical_features] = df[categorical_features].fillna("없음")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

meta_features = preprocessor.fit_transform(df)

###최종 임베딩 결합 (img+메타데이터)

In [39]:
final_embeddings = []

for idx, row in df.iterrows():
    meta_vec = meta_features[idx].toarray().flatten() if hasattr(meta_features[idx], 'toarray') else meta_features[idx]

    if row['has_image'] and row['embedding'] is not None:
        img_vec = row['embedding']
        combined_vec = np.concatenate([img_vec, meta_vec])
    else:
        # 이미지 없는 경우: 0벡터 패딩
        zero_img_vec = np.zeros(512)
        combined_vec = np.concatenate([zero_img_vec, meta_vec])

    final_embeddings.append(combined_vec)

In [40]:
df['final_embedding'] = final_embeddings

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52254 entries, 0 to 52253
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            52254 non-null  int64  
 1   title               52254 non-null  object 
 2   price               52254 non-null  object 
 3   address             52254 non-null  object 
 4   floor               36047 non-null  float64
 5   deposit_type        52254 non-null  object 
 6   management_fee      42877 non-null  float64
 7   available_from      52254 non-null  object 
 8   house_num           52254 non-null  int64  
 9   agent_comm          52078 non-null  float64
 10  agent_info          52254 non-null  object 
 11  rooms_count         52146 non-null  float64
 12  options             52254 non-null  object 
 13  posted_at           51882 non-null  object 
 14  gu                  52254 non-null  object 
 15  dong                52254 non-null  object 
 16  img_

###Faiss 인덱스 생성 및 저장

In [43]:
embedding_dim = len(final_embeddings[0])
index = faiss.IndexFlatIP(embedding_dim)

embedding_matrix = np.vstack(final_embeddings).astype('float32')
faiss.normalize_L2(embedding_matrix)
index.add(embedding_matrix)

In [44]:
# Faiss 인덱스 저장
faiss.write_index(index, "faiss_index.faiss")

###유사 매물 추천 함수

In [45]:
def search_similar(query_house_num, top_k=5):
    row_match = df[df['house_num'] == query_house_num]
    if row_match.empty:
        print(f"[❗] house_num={query_house_num} 에 해당하는 매물이 없습니다.")
        return pd.DataFrame()

    row = row_match.iloc[0]

    query_vec = np.array(row['final_embedding'], dtype='float32').reshape(1, -1)
    faiss.normalize_L2(query_vec)

    D, I = index.search(query_vec, top_k)

    result = df.iloc[I[0]].copy()
    result['similarity'] = D[0]
    return result[['house_num', 'title', 'gu', 'dong', 'deposit', 'monthly_rent', 'space', 'similarity']]

In [47]:
# 테스트
result = search_similar(2527626361, top_k=10)
print(result)

     house_num          title   gu dong    deposit  monthly_rent  space  \
0   2527626361         일반원룸1층  광진구  광장동  100000000             0      9   
20  2526202483         일반원룸6층  광진구  광장동  150000000             0     10   
34  2526022193         일반원룸6층  광진구  광장동  150000000             0     10   
63  2522734529         일반원룸6층  광진구  광장동  150000000             0     10   
78  2523327461          다가구저층  광진구  광장동  175000000             0      9   
19  2526554675           빌라4층  광진구  광장동  200000000             0      9   
47  2524056128          다가구2층  광진구  광장동  210000000             0     10   
1   2527936243           빌라고층  광진구  광장동  200000000             0      9   
40  2526695290         일반원룸6층  광진구  광장동  150000000             0      9   
51  2525897118  현대골든텔2 101동7층  광진구  광장동  130000000             0      8   

    similarity  
0            1  
20           1  
34           1  
63           1  
78           1  
19           1  
47           1  
1            1  
40           1  
51  

In [ ]:
# 중복 제거
# title, address, gu, dong, price, area_size 모두 같으면 drop

In [ ]:
# 💡 다음 단계 (선택)
# 	•	중복 매물 제거: title, area_size, floor, price 조합으로 클러스터링
# 	•	API화 (FastAPI)
# 	•	임베딩 캐싱 (npy로 저장)
# 	•	이미지가 유사하면서도 지역, 가격, 평수도 유사한 하이브리드 필터 추가
# 	•	광각사진 탐지 모델 연동

In [48]:
# 벡터 직렬화
df['final_embedding'] = df['final_embedding'].apply(
    lambda x: json.dumps(x.tolist()) if isinstance(x, np.ndarray) else json.dumps(x)
)

In [52]:
df = df.drop(['img_path', 'has_image','local_path', 'embedding'], axis=1)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52254 entries, 0 to 52253
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   house_id            52254 non-null  int64  
 1   title               52254 non-null  object 
 2   price               52254 non-null  object 
 3   address             52254 non-null  object 
 4   floor               36047 non-null  float64
 5   deposit_type        52254 non-null  object 
 6   management_fee      42877 non-null  float64
 7   available_from      52254 non-null  object 
 8   house_num           52254 non-null  int64  
 9   agent_comm          52078 non-null  float64
 10  agent_info          52254 non-null  object 
 11  rooms_count         52146 non-null  float64
 12  options             52254 non-null  object 
 13  posted_at           51882 non-null  object 
 14  gu                  52254 non-null  object 
 15  dong                52254 non-null  object 
 16  img_

In [54]:
import sqlite3

# 데이터베이스 연결
conn = sqlite3.connect('house_with_embedding.db')

# 데이터프레임을 SQLite 테이블로 저장
df.to_sql('house', conn, if_exists='replace', index=False)

# 연결 종료
conn.close()